## Instructions

1.Create a GitHub repository for your interview project, you may name it however you like.  If you would prefer to have a private repository, then you will be asked to share access to your repository and/or submit a ZIP file of your code which includes the git repository.

2. Create all of the necessary project configurations needed for your Python project, you are free to set up the project configurations however you feel is the most appropriate and/or using
whatever tools you prefer. However, it is expected that you provide instructions in the README
for these tools and how to run your code.

3. For the 3 files, using python, Make a copy of the 3 files in a directory Archive/Original.

4. Using Python, You will have to create files called ADT_(TodaysDate)_Modified_file.csv and ORU_(TodaysDate)_Modified_file.csv, the files will be in the directory Archive/Modified.  If there is another file type, you may follow this pattern.  These will be your output files.
a. In the csv, there’s a column called message_type, (ADT/ORU/etc).  
The ADT message type will be in the ADT csv and the ORU message type will be in the ORU csv as mentioned above.

b. Using the 2 text files (ORU and ADT).  Add an extra row for each of the data in there (Please add at least the columns following columns into the output file (the more you add, the better)): Message_type,  patient_first_name, patient_last_name, patient_middle_name, patient_address, state, account_number, and bill_amount (with value of 1234 in the column, the value here should be added to the two files that you insert which are Sample ORU.txt and ADT_sample.txt) 

c. Add another Column called date_of_service, which adds today’s date to the column.

d. Manipulate the column patient_first_name, patient_last_name, patient_middle_name and combine the three columns into one column called patient_name, the format for this column will be “patient_last_name, patient_first_name patient_middle_name”.

e. Create a report file in txt, that lists the total bill amount for each state. (Using bill amount column in the sampledata.csv)


###### Import necessary libraries

In [1]:
import pandas as pd
import shutil 
import hl7
from datetime import date

###### Copy the source files to Archive / Original folder

In [2]:
shutil.copy2('ADT_sample.txt', 'Archive/Original/ADT_sample.txt')
shutil.copy2('Sample ORU.txt', 'Archive/Original/Sample ORU.txt')
shutil.copy2('sampledata.csv', 'Archive/Original/sampledata.csv')

'Archive/Original/sampledata.csv'

###### Load the original sampledata.csv file

In [3]:
csv_data = pd.read_csv('sampledata.csv')

###### Function to append \r to the end of each line so it can be loaded into the h17 library

In [4]:
def r (filename):
    new_adt_msg = ""
    file=open(filename,"r")
    adt_msg = file.readlines()
    for line in adt_msg:
        line = line.replace('\t','')
        line = line.replace('\n','')
        line += '\r'
        new_adt_msg+=line
    return new_adt_msg    

In [5]:
ADT = hl7.parse(r('ADT_sample.txt'))
ORU = hl7.parse(r('Sample ORU.txt'))    

###### Create the new ADT and ORU row from the respective text files

In [6]:
new_adt_row = []
new_adt_row.append(str(ADT.segment('MSH')[9][0][0]) + '-' + str(ADT.segment('MSH')[9][0][1]))
new_adt_row.append(str(ADT.segment('PID')[5][0][0]))
new_adt_row.append(str(ADT.segment('PID')[5][0][1]))
new_adt_row.append(str(ADT.segment('PID')[5][0][2]))
new_adt_row.append(str(ADT.segment('PID')[11][0]).split('^')[0]) # address 1
new_adt_row.append(str(ADT.segment('PID')[11][0]).split('^')[1]) # address 2
new_adt_row.append(str(ADT.segment('PID')[11][0]).split('^')[2]) # city
new_adt_row.append(str(ADT.segment('PID')[11][0]).split('^')[3]) # state
new_adt_row.append(str(ADT.segment('PID')[11][0]).split('^')[4]) # zip
new_adt_row.append(str(ADT.segment('PID')[18][0])) #account number
new_adt_row.append(1234)

In [7]:
new_oru_row = []
new_oru_row.append(str(ORU.segment('MSH')[9][0][0]) + '-' + str(ORU.segment('MSH')[9][0][1]))
new_oru_row.append(str(ORU.segment('PID')[5][0][0]))
new_oru_row.append(str(ORU.segment('PID')[5][0][1]))
new_oru_row.append(str(ORU.segment('PID')[5][0][2]))
new_oru_row.append(str(ORU.segment('PID')[11][0]).split('^')[0]) # address 1
new_oru_row.append(str(ORU.segment('PID')[11][0]).split('^')[1]) # address 2
new_oru_row.append(str(ORU.segment('PID')[11][0]).split('^')[2]) # city
new_oru_row.append(str(ORU.segment('PID')[11][0]).split('^')[3]) # state
new_oru_row.append(str(ORU.segment('PID')[11][0]).split('^')[4]) # zip
new_oru_row.append(str(ORU.segment('PID')[18][0])) #account number
new_oru_row.append(1234)

###### Columns for the new .csv files

In [8]:
lst_cols = ['message_type','patient_first_name','patient_last_name','patient_middle_name',
                 'patient_address_1','patient_address_2','patient_city','patient_state','patient_zip','account_number','bill_amount']

###### Create new ADT and ORU csv files including records from the text files and add new columsns date_of_service and patient_name.

In [9]:
new_adt_ds = csv_data[csv_data['message_type'].str.startswith('ADT')][lst_cols]

In [10]:
new_oru_ds = csv_data[csv_data['message_type'].str.startswith('ORU')][lst_cols]

In [11]:
new_adt_ds.loc[len(new_adt_ds.index)] = new_adt_row

In [12]:
new_oru_ds.loc[len(new_oru_ds.index)] = new_oru_row

In [13]:
new_adt_ds[lst_cols] = new_adt_ds[lst_cols].fillna('').astype(str)

In [14]:
new_oru_ds[lst_cols] = new_oru_ds[lst_cols].fillna('').astype(str)

In [15]:
new_adt_ds['date_of_service'] = date.today()

In [16]:
new_oru_ds['date_of_service'] = date.today()

In [17]:
new_adt_ds['patient_name'] = new_adt_ds['patient_last_name'] + ' ' + new_adt_ds['patient_first_name']  + ' ' + new_adt_ds['patient_middle_name']

In [18]:
new_oru_ds['patient_name'] = new_oru_ds['patient_last_name'] + ' ' + new_oru_ds['patient_first_name']  + ' ' + new_oru_ds['patient_middle_name']

In [19]:
new_adt_ds.to_csv('Archive/Modified/ADT_' + str(date.today()) + '_Modified_file.csv' , index=False)

In [20]:
new_oru_ds.to_csv('Archive/Modified/ORU_' + str(date.today()) + '_Modified_file.csv' , index=False)

In [21]:
rpt = csv_data.groupby('patient_state',as_index = False)[['bill_amount']].sum()
rpt.head()

,patient_state,bill_amount
0,,10364
1,CA,21115
2,IN,64355
3,MO,1597
4,NY,5793


###### Save the report.txt text file

In [22]:
rpt.to_csv('Archive/Modified/report.txt', sep='\t', index=False)

In [43]:
import sqlite3
connection = sqlite3.connect('adt.db')

In [58]:
try:
    connection.execute("DROP TABLE adt_table")
except sqlite3.OperationalError:
    print('No table exists')

No table exists


In [54]:
new_adt_ds.to_sql('adt_table', connection, if_exists='append', index=False)

165

In [55]:
cursor = connection.cursor()
select_all = "SELECT * FROM adt_table"
rows = cursor.execute(select_all).fetchall()

In [56]:
rows

[('ADT-A08',
  'testinggirl',
  'mu',
  '',
  '12345 Address line',
  'SOMEWHERE',
  'VALENCIA',
  'CA',
  '91355',
  'CA00001',
  '700',
  '2023-09-29',
  'mu testinggirl '),
 ('ADT-A08',
  'testinggirl',
  'mu',
  '',
  '12346 Address line',
  'SOMEWHERE',
  'VALENCIA',
  'CA',
  '91355',
  'CA00001',
  '727',
  '2023-09-29',
  'mu testinggirl '),
 ('ADT-A08',
  'testinggirl',
  'mu',
  '',
  '12347 Address line',
  'SOMEWHERE',
  'VALENCIA',
  'CA',
  '91355',
  'CA00001',
  '728',
  '2023-09-29',
  'mu testinggirl '),
 ('ADT-A08',
  'testinggirl',
  'mu',
  '',
  '12348 Address line',
  'SOMEWHERE',
  'VALENCIA',
  'CA',
  '91355',
  'CA00001',
  '912',
  '2023-09-29',
  'mu testinggirl '),
 ('ADT-A04',
  'QINEVAAcuteprone',
  'Printegratedtest',
  'John',
  '12349 Address line',
  '',
  'EVANSVILLE',
  'IN',
  '47714',
  'CA00003',
  '984',
  '2023-09-29',
  'Printegratedtest QINEVAAcuteprone John'),
 ('ADT-A04',
  'INEVAAcuteprtwo',
  'Printegratedtest',
  'John',
  '12350 Addres